In [1]:
import pandas as pd
import os
import numpy as np
import glob

In [7]:
'''Set path variable and initialize the list of files we pull for each subject'''

path_str = 'ashs/atlas_princeton/'

subs_list = glob.glob('ashs/atlas_princeton/*')
subs_list = [i.split('/')[-1] for i in subs_list]

'''Short list to test with'''
#subs_list = ['sub-001', 'sub-025']
''''''

data_list = ['ses-1/final/ses-1_icv.txt','ses-1/final/ses-1_left_multiatlas_raw_volumes.txt','ses-1/final/ses-1_right_multiatlas_raw_volumes.txt']

In [8]:
'''Opens the files for each subject by making 3 tmp dfs. We manipulate the data and index vars to support concat later'''
net_df = pd.DataFrame()
for sub in subs_list:
    
    sub_df = pd.DataFrame()
    
    icv_df = pd.read_csv(os.path.join(path_str,sub,data_list[0]), sep = "\s+", header = None,)
    
    '''The merge statements combine the l/r column and the measure column to keep them seperate later.'''
    left_df = pd.read_csv(os.path.join(path_str,sub,data_list[1]), sep = "\s+", header = None,)
    left_df['merge'] = left_df[1] + '-' + left_df[2]
    
    right_df = pd.read_csv(os.path.join(path_str,sub,data_list[2]), sep = "\s+", header = None,)
    right_df['merge'] = right_df[1] + '-' + right_df[2]

    '''Here we begin making the subjects total df based off the icv df. Setting the index is required for proper concatination later. Inplace means we don't have to reassign the df.'''
    sub_df['Subject'] = [sub]
    sub_df['ICV'] = icv_df[1]
    sub_df.set_index('Subject', inplace=True)
    
    '''L and R df's are made in identical ways. We make lists from the measures and values and transpose them with ".T" from long to tall. I was inserting the subject number,
    but I had to end up making it the index anyway so append would also work.'''
    l_measures = list(left_df['merge'])
    l_measures.insert(0,'Subject')
    l_values = list(left_df[4])
    l_values.insert(0,sub)
    l_df = pd.DataFrame(l_values).T
    l_df.columns = l_measures
    l_df.set_index('Subject', inplace=True)


    r_measures = list(right_df['merge'])
    r_measures.insert(0,'Subject')
    r_values = list(right_df[4])
    r_values.insert(0,sub)
    r_df = pd.DataFrame(r_values).T
    r_df.columns = r_measures
    r_df.set_index('Subject', inplace=True)
    
    '''Concatinate the tmp df's into one subject df.'''
    sub_df = pd.concat([sub_df, l_df, r_df], axis=1)

    '''Here we concatinate the sub and net dfs.'''
    net_df = pd.concat([net_df, sub_df], sort=False)
    
net_df.to_csv('net_princeton_data.csv')